In [1]:
import pandas as pd

# Opening the file
sample_tl_raw = open("src/text data/Sample_Bible_Tagalog.txt").read()

# Printing the Ilokano Raw Data
sample_tl_raw

'GENESIS\nNilalang ng Dios ang sanglibutan.\nNang pasimula ay nilikha ng Dios ang langit at ang lupa.\nAt ang lupa ay walang anyo at walang laman; at ang kadiliman ay sumasa ibabaw ng kalaliman; at ang Espiritu ng Dios ay sumasa ibabaw ng tubig.\nAt sinabi ng Dios Magkaroon ng liwanag; at nagkaroon ng liwanag.\nAt nakita ng Dios ang liwanag na mabuti, at inihiwalay ng Dios ang liwanag sa kadiliman.\nAt tinawag ng Dios ang liwanag na Araw, at tinawag niya ang kadiliman na Gabi. At nagkahapon at nagkaumaga ang unang araw.\nAt sinabi ng Dios, Magkaroon ng isang kalawakan sa gitna ng tubig, at mahiwalay ang tubig sa kapuwa tubig.\nAt ginawa ng Dios ang kalawakan, at inihiwalay ang tubig na nasa ilalim ng kalawakan, sa tubig na nasa itaas ng kalawakan: at nagkagayon.\nAt tinawag ng Dios ang kalawakan na Langit. At nagkahapon at nagkaumaga ang ikalawang araw.\nAt sinabi ng Dios, Mapisan ang tubig na nasa silong ng langit sa isang dako, at lumitaw ang katuyuan, at nagkagayon.\nAt tinawag ng D

In [2]:
# Splitting the raw data into sentences
parsed_sp_tl_raw = sample_tl_raw.split("\n")

parsed_sp_tl_raw 

['GENESIS',
 'Nilalang ng Dios ang sanglibutan.',
 'Nang pasimula ay nilikha ng Dios ang langit at ang lupa.',
 'At ang lupa ay walang anyo at walang laman; at ang kadiliman ay sumasa ibabaw ng kalaliman; at ang Espiritu ng Dios ay sumasa ibabaw ng tubig.',
 'At sinabi ng Dios Magkaroon ng liwanag; at nagkaroon ng liwanag.',
 'At nakita ng Dios ang liwanag na mabuti, at inihiwalay ng Dios ang liwanag sa kadiliman.',
 'At tinawag ng Dios ang liwanag na Araw, at tinawag niya ang kadiliman na Gabi. At nagkahapon at nagkaumaga ang unang araw.',
 'At sinabi ng Dios, Magkaroon ng isang kalawakan sa gitna ng tubig, at mahiwalay ang tubig sa kapuwa tubig.',
 'At ginawa ng Dios ang kalawakan, at inihiwalay ang tubig na nasa ilalim ng kalawakan, sa tubig na nasa itaas ng kalawakan: at nagkagayon.',
 'At tinawag ng Dios ang kalawakan na Langit. At nagkahapon at nagkaumaga ang ikalawang araw.',
 'At sinabi ng Dios, Mapisan ang tubig na nasa silong ng langit sa isang dako, at lumitaw ang katuyuan, 

In [3]:
dict_sm_tl = pd.DataFrame(parsed_sp_tl_raw, columns = ['Sentence'])

# Printing the first 5 rows of the DataFrame
dict_sm_tl.head()

,Sentence
0,GENESIS
1,Nilalang ng Dios ang sanglibutan.
2,Nang pasimula ay nilikha ng Dios ang langit at...
3,At ang lupa ay walang anyo at walang laman; at...
4,At sinabi ng Dios Magkaroon ng liwanag; at nag...


## Viterbi Algorithm

In [4]:
# Opening the determiners file
tl_determiners = open("src/text data/TL_Determiners.txt").read()

tl_determiners

'ng\nang\nNang\nay\nmga\nna\nsa'

In [5]:
# Parsing the determiners file
parsed_tl_dtmn = tl_determiners.split("\n")

parsed_tl_dtmn

['ng', 'ang', 'Nang', 'ay', 'mga', 'na', 'sa']

## Cleaning the Data

In [6]:
import string

def remove_punct(pText):
    text_nopumct = "".join([char for char in pText if char not in string.punctuation])
    return text_nopumct

cleaned_sp_tl = [remove_punct(word) for word in parsed_sp_tl_raw]

cleaned_sp_tl[0:500]

['GENESIS',
 'Nilalang ng Dios ang sanglibutan',
 'Nang pasimula ay nilikha ng Dios ang langit at ang lupa',
 'At ang lupa ay walang anyo at walang laman at ang kadiliman ay sumasa ibabaw ng kalaliman at ang Espiritu ng Dios ay sumasa ibabaw ng tubig',
 'At sinabi ng Dios Magkaroon ng liwanag at nagkaroon ng liwanag',
 'At nakita ng Dios ang liwanag na mabuti at inihiwalay ng Dios ang liwanag sa kadiliman',
 'At tinawag ng Dios ang liwanag na Araw at tinawag niya ang kadiliman na Gabi At nagkahapon at nagkaumaga ang unang araw',
 'At sinabi ng Dios Magkaroon ng isang kalawakan sa gitna ng tubig at mahiwalay ang tubig sa kapuwa tubig',
 'At ginawa ng Dios ang kalawakan at inihiwalay ang tubig na nasa ilalim ng kalawakan sa tubig na nasa itaas ng kalawakan at nagkagayon',
 'At tinawag ng Dios ang kalawakan na Langit At nagkahapon at nagkaumaga ang ikalawang araw',
 'At sinabi ng Dios Mapisan ang tubig na nasa silong ng langit sa isang dako at lumitaw ang katuyuan at nagkagayon',
 'At tin

In [7]:
import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens


tokenized_sp_tl = [tokenize(word) for word in parsed_sp_tl_raw]

dict_sm_tl['Tokenized'] = tokenized_sp_tl
dict_sm_tl.head()

,Sentence,Tokenized
0,GENESIS,[GENESIS]
1,Nilalang ng Dios ang sanglibutan.,"[Nilalang, ng, Dios, ang, sanglibutan, ]"
2,Nang pasimula ay nilikha ng Dios ang langit at...,"[Nang, pasimula, ay, nilikha, ng, Dios, ang, l..."
3,At ang lupa ay walang anyo at walang laman; at...,"[At, ang, lupa, ay, walang, anyo, at, walang, ..."
4,At sinabi ng Dios Magkaroon ng liwanag; at nag...,"[At, sinabi, ng, Dios, Magkaroon, ng, liwanag,..."


In [8]:
def check_dtmn(sentence, dtmn_list):
    """
    This function checks if the specific word in the sentence is a determiner, and extracts it.
    """
    text = [word for word in sentence if word in dtmn_list]
    return text


dict_sm_tl['Determiner'] = dict_sm_tl['Tokenized'].apply(lambda x: check_dtmn(x, parsed_tl_dtmn))

dict_sm_tl.head()

,Sentence,Tokenized,Determiner
0,GENESIS,[GENESIS],[]
1,Nilalang ng Dios ang sanglibutan.,"[Nilalang, ng, Dios, ang, sanglibutan, ]","[ng, ang]"
2,Nang pasimula ay nilikha ng Dios ang langit at...,"[Nang, pasimula, ay, nilikha, ng, Dios, ang, l...","[Nang, ay, ng, ang, ang]"
3,At ang lupa ay walang anyo at walang laman; at...,"[At, ang, lupa, ay, walang, anyo, at, walang, ...","[ang, ay, ang, ay, ng, ang, ng, ay, ng]"
4,At sinabi ng Dios Magkaroon ng liwanag; at nag...,"[At, sinabi, ng, Dios, Magkaroon, ng, liwanag,...","[ng, ng, ng]"


In [9]:
temp_word = "kumain at tumakbo yung bata"
panlapi = ["um", "in"]

# checking if the word contains the suffixes


"""
if temp_word.__contains__("um"):
    print("The word contains the sufix 'um'")
"""

for suffix in panlapi:
    verb = [word for word in temp_word.split(" ") if word.__contains__(suffix)]
    
verb
# verb = [word for word in temp_word.split(" ") if word.__contains__("um")]
# verb
    

['kumain']

In [10]:
""" 
    Affixes
"""
PREFIX_SET = [
    'nakikipag', 'pakikipag',
    'pinakama', 'pagpapa',
    'pinagka', 'panganga',
    'makapag', 'nakapag',
    'tagapag', 'makipag',
    'nakipag', 'tigapag',
    'pakiki', 'magpa',
    'napaka', 'pinaka',
    'ipinag', 'pagka',
    'pinag', 'mapag',
    'mapa', 'taga',
    'ipag', 'tiga',
    'pala', 'pina',
    'pang', 'naka',
    'nang', 'mang',
    'sing',
    'ipa', 'pam',
    'pan', 'pag',
    'tag', 'mai',
    'mag', 'nam',
    'nag', 'man',
    'may', 'ma',
    'na', 'ni',
    'pa', 'ka',
    'um', 'in',
    'i',
]

INFIX_SET = [
    'um', 'in',
]

SUFFIX_SET = [
    'syon','dor',
    'ita', 'han',
    'hin', 'ing',
    'ang', 'ng',
    'an', 'in',
    'g',
]

In [31]:
def check_verb(sentence, prefix_list, infix_list, suffix_list, dtmn_list):
    """
    This function checks if the specific word in the sentence is a verb, and extracts it.
    """
    
    """
    text = [word for word in sentence if word in verb_list]
    """
    text = []
    
    
    for word in sentence:
        print(word)
        done = False
        
        if word not in dtmn_list:
            for prefix in prefix_list:
                if word.startswith(prefix) & (done == False):
                    text.append(word)
                    done = True
            for infix in infix_list:
                if word.__contains__(infix) & (done == False):
                    text.append(word)
                    done = True
            for suffix in suffix_list:
                if word.endswith(suffix) & (done == False):
                    text.append(word)
                    done = True
                
    return text
# end of function


dict_sm_tl['Verb'] = dict_sm_tl['Tokenized'].apply(lambda x: check_verb(x, PREFIX_SET, INFIX_SET, SUFFIX_SET, parsed_tl_dtmn))
dict_sm_tl.head()

GENESIS
Nilalang
ng
Dios
ang
sanglibutan

Nang
pasimula
ay
nilikha
ng
Dios
ang
langit
at
ang
lupa

At
ang
lupa
ay
walang
anyo
at
walang
laman
at
ang
kadiliman
ay
sumasa
ibabaw
ng
kalaliman
at
ang
Espiritu
ng
Dios
ay
sumasa
ibabaw
ng
tubig

At
sinabi
ng
Dios
Magkaroon
ng
liwanag
at
nagkaroon
ng
liwanag

At
nakita
ng
Dios
ang
liwanag
na
mabuti
at
inihiwalay
ng
Dios
ang
liwanag
sa
kadiliman

At
tinawag
ng
Dios
ang
liwanag
na
Araw
at
tinawag
niya
ang
kadiliman
na
Gabi
At
nagkahapon
at
nagkaumaga
ang
unang
araw

At
sinabi
ng
Dios
Magkaroon
ng
isang
kalawakan
sa
gitna
ng
tubig
at
mahiwalay
ang
tubig
sa
kapuwa
tubig

At
ginawa
ng
Dios
ang
kalawakan
at
inihiwalay
ang
tubig
na
nasa
ilalim
ng
kalawakan
sa
tubig
na
nasa
itaas
ng
kalawakan
at
nagkagayon

At
tinawag
ng
Dios
ang
kalawakan
na
Langit
At
nagkahapon
at
nagkaumaga
ang
ikalawang
araw

At
sinabi
ng
Dios
Mapisan
ang
tubig
na
nasa
silong
ng
langit
sa
isang
dako
at
lumitaw
ang
katuyuan
at
nagkagayon

At
tinawag
ng
Dios
ang
katuyuan
na
Lupa
at

,Sentence,Tokenized,Determiner,Verb
0,GENESIS,[GENESIS],[],[]
1,Nilalang ng Dios ang sanglibutan.,"[Nilalang, ng, Dios, ang, sanglibutan, ]","[ng, ang]","[Nilalang, sanglibutan]"
2,Nang pasimula ay nilikha ng Dios ang langit at...,"[Nang, pasimula, ay, nilikha, ng, Dios, ang, l...","[Nang, ay, ng, ang, ang]","[pasimula, nilikha]"
3,At ang lupa ay walang anyo at walang laman; at...,"[At, ang, lupa, ay, walang, anyo, at, walang, ...","[ang, ay, ang, ay, ng, ang, ng, ay, ng]","[walang, walang, laman, kadiliman, sumasa, iba..."
4,At sinabi ng Dios Magkaroon ng liwanag; at nag...,"[At, sinabi, ng, Dios, Magkaroon, ng, liwanag,...","[ng, ng, ng]","[sinabi, liwanag, nagkaroon, liwanag]"
